<a href="https://colab.research.google.com/github/irasin/jp_w2v/blob/master/jp_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Colabの使い方



*   各ブロックにコードやmarkdown形式のテキストデータが記されている
*   各ブロックの左側の開始ボタンを押すか、クリックしてshift+enterを同時に押すかでブロックを実行できる

詳しく知りたい人は、次のページを参照してください

https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d

https://qiita.com/tomo_makes/items/b3c60b10f7b25a0a5935

https://qiita.com/shoji9x9/items/0ff0f6f603df18d631ab

## Chainerのバージョンを更新
最初に実行するとき、下記のようなWARNINGが出現したら、`ランタイム/ランタイムを再起動`を選んで、再起動する。


WARNING: The following packages were previously imported in this runtime:

 　 [typing]

You must restart the runtime in order to use newly installed versions.

In [1]:
!pip install -U chainer

Requirement already up-to-date: chainer in /usr/local/lib/python3.6/dist-packages (6.2.0)


## GPUが使えるように、対応するCupyのバージョンを更新

In [4]:
!pip install 'cupy-cuda100>=6.2.0,<7.0.0'

## Word2Vecの実装

下記のブロックを実行する際に、Cupyのバージョンが合わないエラーが出たら、上記のCupyの更新をもう一度実行してください。

In [0]:
import argparse
import collections
import os
import six
import warnings

import numpy as np

import chainer
from chainer.backends import cuda
import chainer.functions as F
import chainer.initializers as I
import chainer.links as L
import chainer.optimizers as O
from chainer import reporter
from chainer import training
from chainer.training import extensions


class ContinuousBoW(chainer.Chain):
    """Definition of Continuous Bag of Words Model"""

    def __init__(self, n_vocab, n_units, loss_func):
        super(ContinuousBoW, self).__init__()

        with self.init_scope():
            self.embed = L.EmbedID(
                n_vocab, n_units, initialW=I.Uniform(1. / n_units))
            self.loss_func = loss_func

    def forward(self, x, contexts):
        e = self.embed(contexts)
        h = F.sum(e, axis=1) * (1. / contexts.shape[1])
        loss = self.loss_func(h, x)
        reporter.report({'loss': loss}, self)
        return loss


class SkipGram(chainer.Chain):
    """Definition of Skip-gram Model"""

    def __init__(self, n_vocab, n_units, loss_func):
        super(SkipGram, self).__init__()

        with self.init_scope():
            self.embed = L.EmbedID(
                n_vocab, n_units, initialW=I.Uniform(1. / n_units))
            self.loss_func = loss_func

    def forward(self, x, contexts):
        e = self.embed(contexts)
        batch_size, n_context, n_units = e.shape
        x = F.broadcast_to(x[:, None], (batch_size, n_context))
        e = F.reshape(e, (batch_size * n_context, n_units))
        x = F.reshape(x, (batch_size * n_context,))
        loss = self.loss_func(e, x)
        reporter.report({'loss': loss}, self)
        return loss


class SoftmaxCrossEntropyLoss(chainer.Chain):
    """Softmax cross entropy loss function preceded by linear transformation.
    """

    def __init__(self, n_in, n_out):
        super(SoftmaxCrossEntropyLoss, self).__init__()
        with self.init_scope():
            self.out = L.Linear(n_in, n_out, initialW=0)

    def forward(self, x, t):
        return F.softmax_cross_entropy(self.out(x), t)


class WindowIterator(chainer.dataset.Iterator):
    """Dataset iterator to create a batch of sequences at different positions.
    This iterator returns a pair of the current words and the context words.
    """

    def __init__(self, dataset, window, batch_size, repeat=True):
        self.dataset = np.array(dataset, np.int32)
        self.window = window  # size of context window
        self.batch_size = batch_size
        self._repeat = repeat
        # order is the array which is shuffled ``[window, window + 1, ...,
        # len(dataset) - window - 1]``
        self.order = np.random.permutation(
            len(dataset) - window * 2).astype(np.int32)
        self.order += window
        self.current_position = 0
        # Number of completed sweeps over the dataset. In this case, it is
        # incremented if every word is visited at least once after the last
        # increment.
        self.epoch = 0
        # True if the epoch is incremented at the last iteration.
        self.is_new_epoch = False

    def __next__(self):
        """This iterator returns a list representing a mini-batch.
        Each item indicates a different position in the original sequence.
        """
        if not self._repeat and self.epoch > 0:
            raise StopIteration

        i = self.current_position
        i_end = i + self.batch_size
        position = self.order[i:i_end]
        w = np.random.randint(self.window - 1) + 1
        offset = np.concatenate([np.arange(-w, 0), np.arange(1, w + 1)])
        pos = position[:, None] + offset[None, :]
        contexts = self.dataset.take(pos)
        center = self.dataset.take(position)

        if i_end >= len(self.order):
            np.random.shuffle(self.order)
            self.epoch += 1
            self.is_new_epoch = True
            self.current_position = 0
        else:
            self.is_new_epoch = False
            self.current_position = i_end

        return center, contexts

    @property
    def epoch_detail(self):
        return self.epoch + float(self.current_position) / len(self.order)

    def serialize(self, serializer):
        self.current_position = serializer('current_position',
                                           self.current_position)
        self.epoch = serializer('epoch', self.epoch)
        self.is_new_epoch = serializer('is_new_epoch', self.is_new_epoch)
        if self.order is not None:
            serializer('order', self.order)


@chainer.dataset.converter()
def convert(batch, device):
    center, contexts = batch
    center = device.send(center)
    contexts = device.send(contexts)
    return center, contexts

## Google Driveの認証
実行した後、認証用のリンクがあるので、自分のGoogleアカウントで認証してください。
認証コードを下のブランクに貼って確認する

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## 自分のドライブの中身を確認する

tokenized_data.txtがあるはず



In [9]:
!ls -l drive/My\ Drive/ 

total 836831
drwx------ 2 root root      4096 Jun  3 11:51  コンペ2018
drwx------ 2 root root      4096 May 16 04:52  Avatar-Net-Pytorch
-rw------- 1 root root     83492 Jul 21 11:35  c2v.ipynb
drwx------ 2 root root      4096 Dec 14  2018 'Colab Notebooks'
drwx------ 2 root root      4096 May 20  2018  miku
-rw------- 1 root root   7132316 Aug  5 06:49  p1780-tang.pdf
drwx------ 2 root root      4096 Aug  5 06:40  p2v_src
drwx------ 2 root root      4096 May 20  2018  programming
drwx------ 2 root root      4096 May 21 01:19  randd_eight_skill_prediction
drwx------ 2 root root      4096 May  8 07:49  StyleTransfer
drwx------ 2 root root      4096 Jan 15  2018  tex資料
-rw------- 1 root root  57887962 Aug  6 15:03  tokenized_data.txt
drwx------ 2 root root      4096 Jun 14 08:42  VISA更新
-rw------- 1 root root  58070450 Aug  6 10:42  wiki50_tokenzied.txt
-rw------- 1 root root 733469461 Aug  6 11:21  word2vec.model
drwx------ 2 root root      4096 Feb 18 08:47  就活
drwx------ 2 root root    

## tokenized_data.txtに対して、Word2Vecの学習を行うための前処理を行う

In [0]:
with open('drive/My Drive/tokenized_data.txt', 'r') as f:
    data = f.read().split()

import collections
import numpy as np

index2word = {}
word2index = {}


idx = 0
for w in set(data):
    word2index[w] = idx
    index2word[idx] = w
    idx += 1

data_array = []

for w in data:
    data_array.append(word2index[w])

data_array = np.array(data_array, dtype='int32')

In [0]:
# Set up the dataset
train = data_array[:]

counts = collections.Counter(train)
n_vocab = max(train) + 1

vocab = word2index  

In [0]:
parser = argparse.ArgumentParser()
parser.add_argument('--device', '-d', type=str, default='0',
                    help='Device specifier. Either ChainerX device '
                    'specifier or an integer. If non-negative integer, '
                    'CuPy arrays with specified device id are used. If '
                    'negative integer, NumPy arrays are used')
parser.add_argument('--unit', '-u', default=200, type=int,
                    help='number of units')
parser.add_argument('--window', '-w', default=10, type=int,
                    help='window size')
parser.add_argument('--batchsize', '-b', type=int, default=1000,
                    help='learning minibatch size')
parser.add_argument('--epoch', '-e', default=3, type=int,
                    help='number of epochs to learn')
parser.add_argument('--model', '-m', choices=['skipgram', 'cbow'],
                    default='skipgram',
                    help='model type ("skipgram", "cbow")')
parser.add_argument('--negative-size', default=5, type=int,
                    help='number of negative samples')
parser.add_argument('--out-type', '-o', choices=['hsm', 'ns', 'original'],
                    default='hsm',
                    help='output model type ("hsm": hierarchical softmax, '
                    '"ns": negative sampling, "original": '
                    'no approximation)')
parser.add_argument('--out', default='result',
                    help='Directory to output the result')
group = parser.add_argument_group('deprecated arguments')
group.add_argument('--gpu', '-g', dest='device',
                   type=int, nargs='?', const=0,
                   help='GPU ID (negative value indicates CPU)')
args = parser.parse_args(args=[])

if chainer.get_dtype() == np.float16:
    warnings.warn(
        'This example may cause NaN in FP16 mode.', RuntimeWarning)

device = chainer.get_device(args.device)
device.use()

    
print('Device: {}'.format(device))
print('# unit: {}'.format(args.unit))
print('Window: {}'.format(args.window))
print('Minibatch-size: {}'.format(args.batchsize))
print('# epoch: {}'.format(args.epoch))
print('Training model: {}'.format(args.model))
print('Output type: {}'.format(args.out_type))
print('')

print('n_vocab: %d' % n_vocab)
print('data length: %d' % len(train))

if args.out_type == 'hsm':
    HSM = L.BinaryHierarchicalSoftmax
    tree = HSM.create_huffman_tree(counts)
    loss_func = HSM(args.unit, tree)
    loss_func.W.array[...] = 0   
elif args.out_type == 'ns':
    cs = [counts[w] for w in range(len(counts))]
    loss_func = L.NegativeSampling(args.unit, cs, args.negative_size)
    loss_func.W.array[...] = 0
elif args.out_type == 'original':
    loss_func = SoftmaxCrossEntropyLoss(args.unit, n_vocab)
else:
    raise Exception('Unknown output type: {}'.format(args.out_type))

# Choose the model
if args.model == 'skipgram':
    model = SkipGram(n_vocab, args.unit, loss_func)
    
elif args.model == 'cbow':
    model = ContinuousBoW(n_vocab, args.unit, loss_func)
    
else:
    raise Exception('Unknown model type: {}'.format(args.model))

model.to_device(device)

# Set up an optimizer
optimizer = O.Adam()
optimizer.setup(model)

# Set up an iterator
train_iter = WindowIterator(train, args.window, args.batchsize)
val_iter = WindowIterator(val, args.window, args.batchsize, repeat=False)

# Set up an updater
updater = training.updaters.StandardUpdater(
    train_iter, optimizer, converter=convert, device=device)

# Set up a trainer
trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=args.out)
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss']))
trainer.extend(extensions.ProgressBar())

trainer.run()



In [0]:
# Save the word2vec model
with open('word2vec.model', 'w') as f:
    f.write('%d %d\n' % (len(index2word), args.unit))
    w = cuda.to_cpu(model.embed.W.array)
    for i, wi in enumerate(w):
        v = ' '.join(map(str, wi))
        f.write('%s %s\n' % (index2word[i], v))

## word2vec.modelの存在を確認

In [16]:
!ls -l 

total 828600
drwx------ 3 root root      4096 Aug 10 09:01 drive
drwxr-xr-x 2 root root      4096 Aug 10 09:19 result
drwxr-xr-x 1 root root      4096 Aug  2 16:06 sample_data
-rw-r--r-- 1 root root 848469987 Aug 10 09:20 word2vec.model


## 自分のGoogle Driveに学習済みのword2vec.modelを保存
ここでもう一度認証をする必要がある

In [0]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [18]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'word2vec.model',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('word2vec.model', 
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1synbt6xY-Tp68yAYQV9PURBEM9p3yXoc


## 学習済みのword2vec.modelを試してみる
下記のコードはGoogle Driveからword2vec.modelをローカルにダウンロードして、test.pyを実行しても良い

UserWarningは無視して大丈夫

In [19]:
import logging
from gensim.models import KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logging.root.setLevel(level=logging.INFO)


model = KeyedVectors.load_word2vec_format('word2vec.model', binary=False)

2019-08-10 09:21:17,215 : INFO : loading projection weights from word2vec.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-08-10 09:22:21,640 : INFO : loaded (361995, 200) matrix from word2vec.model


In [20]:
model.most_similar(positive=["トヨタ"], topn=10)

2019-08-10 09:22:23,760 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('レクサス', 0.7090200781822205),
 ('日産', 0.7048579454421997),
 ('スバル', 0.6470560431480408),
 ('軽自動車', 0.6420823335647583),
 ('四輪駆動', 0.6304976940155029),
 ('ダットサン', 0.6265804171562195),
 ('ブランド', 0.6081885695457458),
 ('4WD', 0.6024623513221741),
 ('高級車', 0.6004769802093506),
 ('フォルクスワーゲン', 0.5939617156982422)]

In [21]:
model.most_similar(positive=["声優"], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('タレント', 0.7015098929405212),
 ('ナレーター', 0.6986377239227295),
 ('俳優', 0.6952576637268066),
 ('女優', 0.6380811333656311),
 ('脚本家', 0.6325925588607788),
 ('ラジオパーソナリティ', 0.6181122064590454),
 ('お笑い芸人', 0.6018644571304321),
 ('歌手', 0.5958029627799988),
 ('漫画家', 0.5934780836105347),
 ('ナレーション', 0.5912138223648071)]

In [22]:
model.most_similar(positive=["Python"], topn=10)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('C言語', 0.7063682079315186),
 ('プログラミング言語', 0.6869475841522217),
 ('Java', 0.6586480736732483),
 ('型付き', 0.6450315713882446),
 ('HTML', 0.6415669918060303),
 ('ライブラリ', 0.6411113739013672),
 ('インタプリタ', 0.6388726234436035),
 ('Perl', 0.6275331377983093),
 ('C++', 0.6211255788803101),
 ('実装', 0.6199963092803955)]